### TESTING 

In [109]:
import pandas as pd
from sklearn.ensemble import IsolationForest
import math
import plotly.graph_objects as go

In [110]:
filename = 'all_combined_EMA2227A10005.csv'
df = pd.read_csv('data/'+filename)
df = df.dropna(subset='cycle_time')
# df = df.rename(columns={'CT': 'cycle_time', 'Shot Time': 'shot_time'})

In [54]:
df = df.dropna(subset='cycle_time')
df = df[['shot_time','cycle_time']]
data = ['cycle_time']
df['cycle_time'] = df['cycle_time'].apply(math.floor)
if_model = IsolationForest(contamination='auto',random_state=42)
if_model.fit(df[data])
df['anomoly_score'] = if_model.decision_function(df[data])
df['anomoly_score_inverse']=if_model.score_samples(df[data])
# df['anomoly_score'] = df['anomoly_score']+1
df['anomoly_score_inverse'] = df['anomoly_score_inverse']+1
df['output'] = if_model.predict(df[data])

In [55]:
# Define a function to apply to each row
def assign_anomaly_score(row):
    if row['anomoly_score_inverse']<  0.3:
        return -1
    else:
        return 1

# Apply the function to create the new column
df['output_anomaly_score'] = df.apply(assign_anomaly_score, axis=1)

In [56]:
df.to_csv('result/'+filename)

In [259]:
# df['gradient_fwd'] = df['cycle_time'].diff(1)
# df['gradient_bck'] = df['cycle_time'].diff(-1)

In [ ]:
df['gradient_bck'] = df['cycle_time'].diff(-1) # This will fill the first value with zero, because the difference is x = x[i] - x[i+1]
df['gradient_bck'] = df['gradient_bck'].abs()
df['gradient_fwd'] = df['cycle_time'].diff()
df['gradient_fwd'] = df['gradient_fwd'].abs()
tolerance = 5
df.loc[ (df['gradient_fwd'] >= tolerance) & (df['gradient_bck'] >= tolerance), 'output_anomaly_score'] = -1
df

### FOR CONCATENATED DATA

In [113]:
# Get list of files in folder
folder_path = 'data'  # Change this to your folder path
dfs = []  # List to hold DataFrames from each file
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # Assuming all files are CSVs
        file_path = os.path.join(folder_path, filename)
        
        # Read CSV into DataFrame
        df = pd.read_csv(file_path)
        
        # Append DataFrame to list
        dfs.append(df)

# Concatenate all DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)

# Drop rows with NaN values in 'cycle_time' column
combined_df = combined_df.dropna(subset=['cycle_time'])
combined_df.to_csv('combined_data.csv')
# Select only 'shot_time' and 'cycle_time' columns
combined_df = combined_df[['shot_time', 'cycle_time','COUNTER_ID']]

# Apply floor function to 'cycle_time' column
combined_df['cycle_time'] = combined_df['cycle_time'].apply(math.floor)

# Define features
data = ['cycle_time']

# Initialize Isolation Forest model
# if_model = IsolationForest(contamination='auto', random_state=42)
if_model = IsolationForest(contamination=0.1, random_state=42)


# Fit model
if_model.fit(combined_df[data])

# Calculate anomaly scores
combined_df['anomoly_score'] = if_model.decision_function(combined_df[data])
combined_df['anomoly_score_inverse'] = if_model.score_samples(combined_df[data])
combined_df['anomoly_score_inverse'] = combined_df['anomoly_score_inverse'] + 1

# Predict anomalies
combined_df['output'] = if_model.predict(combined_df[data])

# Apply the function to create the new column 'output_anomaly_score'
combined_df['output_anomaly_score'] = combined_df.apply(assign_anomaly_score, axis=1)
post_preprocessing = False
if post_preprocessing == True:
    combined_df['gradient_bck'] = combined_df['cycle_time'].diff(-1) # This will fill the first value with zero, because the difference is x = x[i] - x[i+1]
    combined_df['gradient_bck'] = combined_df['gradient_bck'].abs()
    combined_df['gradient_fwd'] = combined_df['cycle_time'].diff()
    combined_df['gradient_fwd'] = combined_df['gradient_fwd'].abs()

    tolerance = 5
    combined_df.loc[ (combined_df['gradient_fwd'] >= tolerance) & (combined_df['gradient_bck'] >= tolerance), 'output'] = -1

    threhold = 2
    combined_df.loc[ (combined_df['gradient_fwd'] <= threhold) & (combined_df['gradient_bck'] <= threhold), 'output'] = 1

# combined_df.loc[ (combined_df['gradient_fwd'] <= threhold) & (combined_df['gradient_bck'] <= threhold)& (combined_df['output'] == -1), 'output'] = 1
# Save result to a new CSV file
result_folder = 'result'  # Folder to save result
os.makedirs(result_folder, exist_ok=True)  # Create folder if it doesn't exist
result_file_path = os.path.join(result_folder, 'combined_result_iftihar.csv')
combined_df.to_csv(result_file_path, index=False)

C:\Users\uikra\AppData\Local\Temp\ipykernel_22772\3559514974.py:9: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.



In [116]:
import os
from itertools import product
import os
import gc  # Importing garbage collection module
from itertools import product
import json
import pandas as pd
import math
from tqdm import tqdm
from sklearn.ensemble import IsolationForest
# Get list of files in folder
folder_path = 'data'  # Change this to your folder path
dfs = []  # List to hold DataFrames from each file
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # Assuming all files are CSVs
        file_path = os.path.join(folder_path, filename)
        
        # Read CSV into DataFrame
        df = pd.read_csv(file_path)
        
        # Append DataFrame to list
        dfs.append(df)

# Concatenate all DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)

# Drop rows with NaN values in 'cycle_time' column
combined_df = combined_df.dropna(subset=['cycle_time'])
combined_df.to_csv('combined_data.csv')
# Select only 'shot_time' and 'cycle_time' columns
combined_df = combined_df[['shot_time', 'cycle_time','COUNTER_ID']]

# Apply floor function to 'cycle_time' column
combined_df['cycle_time'] = combined_df['cycle_time'].apply(math.floor)

# Define features
data = ['cycle_time']
from tqdm import tqdm

grid_search_dict = {
    'cont_parm': [0.1, 0.2, 0.3, 0.4, 0.5, 'auto'],
    'n_estimators': [50, 100, 150, 200, 250, 300, 'auto'],
    'max_samples': [10, 20, 30, 40, 50, 60, 100, 150, 200, 'auto'],
    'max_features': [1, 0.5, 0.8, 1, 2, 3, 'auto'],
    'bootstrap': [True, False],
    'warm_start': [True, False]
}

param_combinations = list(product(*grid_search_dict.values()))

for comb in tqdm(param_combinations, desc="Generating Files", unit="combination"):
    cont_parm, n_estimators, max_samples, max_features, bootstrap, warm_start = comb
    filename = f"combination_cont{cont_parm}_est{n_estimators}_samp{max_samples}_feat{max_features}_bootstrap{bootstrap}_warmstart{warm_start}"

    try:
 


        # Initialize Isolation Forest model
        # if_model = IsolationForest(contamination='auto', random_state=42)
        # if_model = IsolationForest(contamination=cont_parm, random_state=42)

        if_model = IsolationForest(n_estimators=n_estimators, 
                                   max_samples=max_samples, 
                                   contamination=cont_parm, 
                                   max_features=max_features,
                                    bootstrap=bootstrap, 
                                    random_state=12, 
                                    warm_start=warm_start)
        

        # Fit model
        if_model.fit(combined_df[data])

        # Calculate anomaly scores
        combined_df['anomoly_score'] = if_model.decision_function(combined_df[data])
        combined_df['anomoly_score_inverse'] = if_model.score_samples(combined_df[data])
        combined_df['anomoly_score_inverse'] = combined_df['anomoly_score_inverse'] + 1

        # Predict anomalies
        combined_df[f'output_{cont_parm}'] = if_model.predict(combined_df[data])

        # Apply the function to create the new column 'output_anomaly_score'
        combined_df[f'output_anomaly_score_{cont_parm}'] = combined_df.apply(assign_anomaly_score, axis=1)
        # combined_df.loc[ (combined_df['gradient_fwd'] <= threhold) & (combined_df['gradient_bck'] <= threhold)& (combined_df['output'] == -1), 'output'] = 1
        # Save result to a new CSV file
        result_folder = 'result'  # Folder to save result
        os.makedirs(result_folder, exist_ok=True)  # Create folder if it doesn't exist
        result_file_path = os.path.join(result_folder, f'{filename}.csv')
        combined_df.to_csv(result_file_path, index=False)
        # Perform garbage collection
        gc.collect()
        post_preprocessing = False
        if post_preprocessing == True:
            combined_df['gradient_bck'] = combined_df['cycle_time'].diff(-1) # This will fill the first value with zero, because the difference is x = x[i] - x[i+1]
            combined_df['gradient_bck'] = combined_df['gradient_bck'].abs()
            combined_df['gradient_fwd'] = combined_df['cycle_time'].diff()
            combined_df['gradient_fwd'] = combined_df['gradient_fwd'].abs()

            tolerance = 5
            combined_df.loc[ (combined_df['gradient_fwd'] >= tolerance) & (combined_df['gradient_bck'] >= tolerance), 'output'] = -1

            threhold = 2
            combined_df.loc[ (combined_df['gradient_fwd'] <= threhold) & (combined_df['gradient_bck'] <= threhold), 'output'] = 1


    except Exception as e:
        print(f'Failed filename : {filename}')
        print('Error ->',e)
        continue



C:\Users\uikra\AppData\Local\Temp\ipykernel_22772\2910563235.py:19: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.

Generating Files:   0%|          | 22/11760 [03:45<13:36:50,  4.18s/combination]

Failed filename : combination_cont0.1_est50_samp10_feat2_bootstrapTrue_warmstartTrue
Error -> max_features must be <= n_features
Failed filename : combination_cont0.1_est50_samp10_feat2_bootstrapTrue_warmstartFalse
Error -> max_features must be <= n_features
Failed filename : combination_cont0.1_est50_samp10_feat2_bootstrapFalse_warmstartTrue
Error -> max_features must be <= n_features
Failed filename : combination_cont0.1_est50_samp10_feat2_bootstrapFalse_warmstartFalse
Error -> max_features must be <= n_features
Failed filename : combination_cont0.1_est50_samp10_feat3_bootstrapTrue_warmstartTrue
Error -> max_features must be <= n_features
Failed filename : combination_cont0.1_est50_samp10_feat3_bootstrapTrue_warmstartFalse
Error -> max_features must be <= n_features
Failed filename : combination_cont0.1_est50_samp10_feat3_bootstrapFalse_warmstartTrue
Error -> max_features must be <= n_features
Failed filename : combination_cont0.1_est50_samp10_feat3_bootstrapFalse_warmstartFalse
Erro

Generating Files:   0%|          | 33/11760 [05:06<30:17:39,  9.30s/combination]


KeyboardInterrupt: 

### Iftikhar ki khushi

In [107]:
def calculate_differences(df, column_name):
    for i in range(1, 6):  # Calculate differences up to the fifth order
        # Forward differences
        df[f'diff_fwd_{i}'] = df[column_name].diff(periods=-i)
        df[f'diff_fwd_{i}'] = df[f'diff_fwd_{i}'].abs().round(2)

        # Backward differences
        df[f'diff_bck_{i}'] = df[column_name].diff(periods=i)
        df[f'diff_bck_{i}'] = df[f'diff_bck_{i}'].abs().round(2)

    return df

def create_forward_columns(df, n):
    for i in range(1, n+1):
        df[f'output_fwd_{i}'] = df['output'].shift(-i)
    return df

def create_backward_columns(df, n):
    for i in range(1, n+1):
        df[f'output_bck_{i}'] = df['output'].shift(i)
    return df

def create_five_shots(row):
    fwd_columns = [f'diff_fwd_{i}' for i in range(1, 6)]
    fwd_output_columns = [f'output_fwd_{i}' for i in range(1, 6)]
    bck_columns = [f'diff_bck_{i}' for i in range(1, 6)]
    bck_output_columns = [f'output_bck_{i}' for i in range(1, 6)]
    
    fwd_values = [(row[fwd_columns[i]], row[fwd_output_columns[i]]) for i in range(5)]
    bck_values = [(row[bck_columns[i]], row[bck_output_columns[i]]) for i in range(5)]
    
    return fwd_values + bck_values




In [ ]:
df= pd.read_csv('data/all_combined_EMA2227A10005.csv')
df = df.dropna(subset='cycle_time')
df = df[['shot_time','cycle_time']]
df['output'] = -1
df = calculate_differences(df, 'cycle_time')
df = create_feature_column(df)
df

In [108]:
folder_path = 'data'
dfs = [] 
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # Assuming all files are CSVs
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dfs.append(df)

# Concatenate all DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)

# Drop rows with NaN values in 'cycle_time' column
combined_df = combined_df.dropna(subset=['cycle_time'])

# Select only 'shot_time' and 'cycle_time' columns
combined_df = combined_df[['shot_time', 'cycle_time','COUNTER_ID']]

# Apply floor function to 'cycle_time' column
combined_df['cycle_time'] = combined_df['cycle_time'].apply(math.floor)

# Define features
data = ['cycle_time']

# Initialize Isolation Forest model
if_model = IsolationForest(contamination='auto', random_state=42)

# Fit model
if_model.fit(combined_df[data])

# Calculate anomaly scores
combined_df['anomoly_score'] = if_model.decision_function(combined_df[data])
combined_df['anomoly_score_inverse'] = if_model.score_samples(combined_df[data])
combined_df['anomoly_score_inverse'] = combined_df['anomoly_score_inverse'] + 1
# Predict anomalies
combined_df['output'] = if_model.predict(combined_df[data])
# Apply the function to create the new column 'output_anomaly_score'
combined_df['output_anomaly_score'] = combined_df.apply(assign_anomaly_score, axis=1)
combined_df = calculate_differences(combined_df, 'cycle_time')
tolerance = 5
combined_df.loc[ (combined_df['diff_fwd_1'] >= tolerance) & (combined_df['diff_bck_1'] >= tolerance), 'output'] = -1

threhold = 2
combined_df.loc[ (combined_df['diff_fwd_1'] <= threhold) & (combined_df['diff_bck_1'] <= threhold), 'output'] = 1

# Create forward columns
combined_df = create_forward_columns(combined_df, 5)
# Create backward columns
combined_df = create_backward_columns(combined_df, 5)

combined_df['five_shots'] = combined_df.apply(create_five_shots, axis=1)
result_folder = 'result'  # Folder to save result
os.makedirs(result_folder, exist_ok=True)  # Create folder if it doesn't exist
result_file_path = os.path.join(result_folder, 'combined_result.csv')
combined_df.to_csv(result_file_path, index=False)

C:\Users\uikra\AppData\Local\Temp\ipykernel_22772\3824538710.py:6: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.



In [134]:
def process_file(file_path, output_directory,name):
    data = pd.read_csv(file_path)
    data['shot_time'] = pd.to_datetime(data['shot_time'])
    data = data.dropna(subset=['cycle_time'])
    # Group by COUNTER_ID
    grouped_data = data.groupby('COUNTER_ID')

    for counter_id, group_data in grouped_data:
        # Set color for markers based on output_anomaly_score column
        colors = ['red' if score == -1 else 'rgb(31, 119, 180)' for score in group_data['output_0.1']]

        # Add hover text with anomoly_score_inverse
        hover_text = [f"Cycle Time: {ct}<br>Anomaly Score Inverse: {score_inv}" 
                      for ct, score_inv in zip(group_data['cycle_time'], group_data['anomoly_score_inverse'])]

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=group_data['shot_time'], y=group_data['cycle_time'], 
                                 mode='markers+lines', line=dict(color='darkgrey', width=1), 
                                 marker=dict(symbol='circle', size=8, color=colors),
                                 hoverinfo='text',
                                 hovertext=hover_text))

        fig.update_layout(
            title=f'Cycle Time vs Measurement Date - Counter ID: {counter_id}',
            xaxis_title='Measurement Date',
            yaxis_title='Cycle Time (seconds)',
            template='plotly_white',
            plot_bgcolor='lightgrey',
            xaxis=dict(
                showgrid=True,
                gridcolor='white',
                tickfont=dict(size=14) 
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='white',
                tickfont=dict(size=14)
            )
        )
        # Output file name based on counter_id
        filename = os.path.join(output_directory, f'{name+counter_id}_ct.html')
        fig.write_html(filename)

output_directory = "ct_plots"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

input_directory = "result"
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        name =filename[:-4]
        print(name)
        file_path = os.path.join(input_directory, filename)
        process_file(file_path, output_directory,name)


combination_cont0.1_est50_samp10_feat0.5_bootstrapFalse_warmstartFalse
combination_cont0.1_est50_samp10_feat0.5_bootstrapFalse_warmstartTrue
combination_cont0.1_est50_samp10_feat0.5_bootstrapTrue_warmstartFalse
combination_cont0.1_est50_samp10_feat0.5_bootstrapTrue_warmstartTrue
combination_cont0.1_est50_samp10_feat0.8_bootstrapFalse_warmstartFalse


KeyboardInterrupt: 

### LABELLING PRODUCTION SEAESON WISE

In [8]:
import numpy as np
import math
def convert_strtime_to_datetime(data):
    """
    Converts the 'TFF' column in the given DataFrame to datetime format.

    Parameters:
    data (DataFrame): The input DataFrame containing the 'TFF' column.

    Returns:
    DataFrame: The modified DataFrame with the 'TFF' column converted to datetime format.
    """
    data["TFF"] = pd.to_datetime(data["TFF"], format="%Y%m%d%H%M%S")
    return data


def get_time_difference(data):
    """
    Calculates the time difference between consecutive rows in the 'TFF' column of the given DataFrame.

    Parameters:
    data (DataFrame): The input DataFrame containing the 'TFF' column.

    Returns:
    Series: The time differences between consecutive rows in hours.
    """
    return (data["TFF"] - data["TFF"].shift(1)).dt.total_seconds() / 3600


def get_list_of_prod_seasons(data):
    """
    Splits the given DataFrame into multiple sections based on time differences in the 'TFF' column.

    Parameters:
    data (DataFrame): The input DataFrame containing the 'TFF' column.

    Returns:
    list: A list of DataFrames, where each DataFrame represents a section of consecutive rows with time differences greater than or equal to 4 hours.
    """
    data = data.reset_index()
    data = convert_strtime_to_datetime(data)

    data["time_difference"] = get_time_difference(data)

    break_points = data[
        data["time_difference"] >= 2
    ].index.values
    if break_points.shape[0] > 0:
        list_of_sections_indices = np.sort(
            np.insert(break_points, len(break_points), data.index.values[-1] + 1)
        )
        ps = [
            data.iloc[:x] if i == 0 else data.iloc[list_of_sections_indices[i - 1] : x]
            for i, x in enumerate(list_of_sections_indices)
        ]
    else:
        ps = [data]
    return ps
def assign_anomaly_score(row):
    if row['anomoly_score_inverse']<  0.3:
        return -1
    else:
        return 1
    
def convert_to_desired_format(date_str):
    # Convert string to datetime object
    date_obj = pd.to_datetime(date_str)
    # Convert datetime object to string with the desired format
    formatted_date = date_obj.strftime("%Y%m%d%H%M%S")
    return formatted_date

In [9]:
df = pd.read_csv('all_combined_EMA2233M10296.csv')
df = df.dropna(subset='cycle_time')
df['cycle_time'] = df['cycle_time'].apply(math.floor)
df = df[['shot_time','cycle_time']]
df['TFF'] = df['shot_time'].apply(lambda x: convert_to_desired_format(x))
final_df = pd.DataFrame()
# Get list of production seasons
prod_seasons = get_list_of_prod_seasons(df)

# Process data for each production season
for ps_data in prod_seasons:
    if_model = IsolationForest(contamination='auto',random_state=42)
    if_model.fit(ps_data[['cycle_time']])
    # Calculate anomaly scores
    ps_data['anomaly_score'] = if_model.decision_function(ps_data[['cycle_time']])
    ps_data['anomoly_score_inverse'] = if_model.score_samples(ps_data[['cycle_time']]) + 1
    ps_data['output'] = if_model.predict(ps_data[['cycle_time']])
    ps_data['output_anomaly_score'] = ps_data.apply(assign_anomaly_score, axis=1)
    final_df = pd.concat([final_df, pd.DataFrame(ps_data)], ignore_index=True)
final_df= final_df[['shot_time','cycle_time','anomaly_score','anomoly_score_inverse','output_anomaly_score','output']]
final_df.to_csv('result/labelled_all_combined_EMA2233M10296_2Hr_Th.csv')
final_df

,shot_time,cycle_time,anomaly_score,anomoly_score_inverse,output_anomaly_score,output
0,2022-05-10 19:23:23,68,-0.285656,0.214344,-1,-1
1,2022-05-10 19:24:31,56,0.100849,0.600849,1,1
2,2022-05-10 19:25:28,56,0.100849,0.600849,1,1
3,2022-05-10 19:26:25,56,0.100849,0.600849,1,1
4,2022-05-10 19:27:21,56,0.100849,0.600849,1,1
...,...,...,...,...,...,...
49657,2024-02-26 23:57:13,59,0.076169,0.576169,1,1
49658,2024-02-26 23:58:13,59,0.076169,0.576169,1,1
49659,2024-02-26 23:59:12,59,0.076169,0.576169,1,1
49660,2024-02-27 00:00:12,59,0.076169,0.576169,1,1


### PLOTS

In [317]:
def process_file(file_path, output_directory):
    data = pd.read_csv(file_path)
    data['shot_time'] = pd.to_datetime(data['shot_time'])
    data = data.dropna(subset=['cycle_time'])

    # Set color for markers based on output_anomaly_score column
    colors = ['red' if score == -1 else 'rgb(31, 119, 180)' for score in data['output']]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['shot_time'], y=data['cycle_time'], 
                             mode='markers+lines', line=dict(color='darkgrey', width=1), 
                             marker=dict(symbol='circle', size=8, color=colors)))

    fig.update_layout(
        title='Cycle Time vs Measurement Date ',
        xaxis_title='Measurement Date',
        yaxis_title='Cycle Time (seconds)',
        template='plotly_white',
        plot_bgcolor='lightgrey',
        xaxis=dict(
            showgrid=True,
            gridcolor='white',
            tickfont=dict(size=14) 
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='white',
            tickfont=dict(size=14)
        )
    )

    # Assuming file_path and output_directory are defined
    filename = os.path.join(output_directory, '{}_ct.html'.format(os.path.basename(file_path).split('.')[0]))
    fig.write_html(filename)

output_directory = "ct_plots"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

input_directory = "result"
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(input_directory, filename)
        process_file(file_path, output_directory)


In [118]:
def process_file(file_path, output_directory):
    data = pd.read_csv(file_path)
    data['shot_time'] = pd.to_datetime(data['shot_time'])
    data = data.dropna(subset=['cycle_time'])

    # Set color for markers based on output_anomaly_score column
    colors = ['red' if score == -1 else 'rgb(31, 119, 180)' for score in data['output_0.1']]

    # Add hover text with anomoly_score_inverse
    hover_text = [f"Cycle Time: {ct}<br>Anomaly Score Inverse: {score_inv}" 
                  for ct, score_inv in zip(data['cycle_time'], data['anomoly_score_inverse'])]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['shot_time'], y=data['cycle_time'], 
                             mode='markers+lines', line=dict(color='darkgrey', width=1), 
                             marker=dict(symbol='circle', size=8, color=colors),
                             hoverinfo='text',
                             hovertext=hover_text))

    fig.update_layout(
        title='Cycle Time vs Measurement Date ',
        xaxis_title='Measurement Date',
        yaxis_title='Cycle Time (seconds)',
        template='plotly_white',
        plot_bgcolor='lightgrey',
        xaxis=dict(
            showgrid=True,
            gridcolor='white',
            tickfont=dict(size=14) 
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='white',
            tickfont=dict(size=14)
        )
    )

    # Assuming file_path and output_directory are defined
    filename = os.path.join(output_directory, '{}_ct.html'.format(os.path.basename(file_path).split('.')[0]))
    fig.write_html(filename)

output_directory = "ct_plots"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

input_directory = "result"
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(input_directory, filename)
        process_file(file_path, output_directory)


KeyboardInterrupt: 